## Ejemplo práctico: leer un formulario de un fichero pdf y pasar la información a uno excel

Este ejemplo toma como referencia un hipotético formulario destinado a que un docente recopile información sobre la composición de grupos de trabajo en una clase amplia. Elaborar una base de datos con esta información manualmente puede ser una tarea que ocupe gran cantidad de tiempo si hay un número elevado de estudiantes. Es por ello que se ha decidido repartir un formulario en formato word a los alumnos que deberán entregar al docente en formato pdf. A lo largo de este documento se explica un script sencillo que permite extraer los datos esenciales del formulario y añadirlos a una base de datos en excel.

Las dos siguientes celdas deben ser ejecutadas si los paquetes `tabula` y `openpyxl` no están instalados. En caso contrario se omiten.

In [ ]:
pip install tabula

In [ ]:
pip install openpyxl

Primero se importa la función `read_pdf` del paquete `tabula`, que nos permite extraer tablas directamente de un pdf. Adicionalmente se definen dos funciones que nos servirán posteriormente. Con `get_topic` podemos extraer el tema del trabajo que tendrá el grupo. Esta función nos ayuda a procesar adecuadamente el tema del trabajo cuando éste ocupa más de una línea en el formulario. Por otro lado, `slice_it` agrupa la información de cada alumno.

In [ ]:
from tabula import read_pdf
from os import listdir
from openpyxl import Workbook

def get_topic(df):
    """
    returns the topic of the work
    """
    topic_indexes = [i for i, null in enumerate(df[0].isnull()) if null]
    topic = ' '.join(df[0].iat[i, 1] for i in topic_indexes)
    topic = '{} {}'.format(df[0].columns[1], topic)
    
    return (topic, topic_indexes)

def slice_it(data, s):
    """
    returns a list of size s tuples using elements from data
    """
    new_data = []
    rest = len(data)
    
    i = 0
    while rest > 0:
        new_data.append(list(data[i:i + s]))
        i += s
        rest -= s
    
    return new_data

La siguiente función extrae la tabla del fichero pdf especificado. Devuelve una lista que contiene uno o más `DataFrames`, así que de aquí en adelante se debe trabajar con la sintaxis y los métodos de estos objetos. El fichero debe estar en la misma ubicación que este notebook o, en caso contrario, será necesario modificar `'form.pdf'` con la ruta correcta. A partir de este punto, el procedimiento se basa en depurar esta tabla, ya que tiene muchos caracteres que no queremos, y extraer la información de interés.

En el siguiente fragmento de código se utiliza un objeto `DataFrame`:
* Sobre los [`DataFrames` y el paquete `pandas`](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html)

In [ ]:
df = read_pdf('form1.pdf')

En el siguiente fragmento se emplea `get_topic`. No sólo nos proporciona el tema del trabajo, sino también información útil para limpiar algunas líneas innecesarias de la tabla. Esta limpieza se hace con el método `DataFrame.drop()`. Finalmente, se modifican los nombres de las columnas para facilitar el trabajo y se limpian el resto de filas que no contienen información útil.

En el siguiente fragmento de código se utiliza el método `DataFrame.drop()`:
* Sobre el [método `DataFrame.drop()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html#pandas-dataframe-drop).

In [ ]:
topic, indexes = get_topic(df)

df = df[0].drop(indexes, axis=0)
df.columns = ['A', 'B']
df = df[df['A'] != 'Estudiante']

El siguiente bucle organiza toda la información útil de cada estudiante en un diccionario.

In [ ]:
students = {'Grupo 1':[]}
keys = ('Nombre', 'Apellidos', 'Correo')

for student in slice_it(df['B'], 3):
    students['Grupo 1'].append(student)

¡Veamos el resultado!

In [ ]:
print(students)

Este script sólo recopila los datos de un formulario para ilustrar el proceso de extracción, depuración y selección de información de un fichero pdf. Para ser usado con más ficheros es necesario realizar las siguientes modificaciones. En primer lugar, se identifican las rutas de todos los ficheros pdf que haya en la misma carpeta del Notebook. El método `listdir()` nos aporta una lista completa de los ficheros y únicamente tenemos que comprobar que su nombre termine en `.pdf`. Además, se inicializa `students` de nuevo para evitar errores.

En el siguiente fragmento de código se utilizan el método listdir() y las comprehensiones de lista:
* Sobre el [método `listdir` y el paquete `os`](https://docs.python.org/3.8/library/os.html?highlight=os#os.listdir)
* Sobre las [comprehensiones de lista](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions).

In [ ]:
forms = [file for file in listdir() if file[-4:] == '.pdf']
students = {}

El siguiente fragmento de código es casi una réplica del código que hemos visto antes, pero esta vez se ejecuta una vez por cada fichero pdf que hayamos identificado. Ahora también asignamos el número de grupo a cada estudiante de cada grupo, puesto que es información que necesitamos para nuestra base de datos.

En el siguiente fragmento se utiliza la función enumerate():
* Sobre la [función `enumerate()`]().

In [ ]:
for i, file in enumerate(forms):
    df = read_pdf(file)
    group = 'Grupo {}' + str(i + 1)
    
    # A partir de aquí el código es prácticamente igual.
    topic, indexes = get_topic(df)

    df = df[0].drop(indexes, axis=0)
    df.columns = ['A', 'B']
    df = df[df['A'] != 'Estudiante']

    students[group] = []

    for student in slice_it(df['B'], 3):
        student.append(i + 1)
        students[group].append(student)

print(students)

A continuación se inicializa `Workbook`, que es la clase que nos sirve para crear un nuevo fichero excel, y se añade la cabecera de la base de datos con el método `Worksheet.append()`. Por último, mediante un bucle anidado `for` se añaden todos los alumnos de cada grupo a la base de datos. El método `Worksheet.save()` nos sirve para guardar nuestra base de datos.

En el siguiente fragmento se utiliza la clase Workbook del paquete openpyxl y se emplea un bucle anidado:
* Sobre la [clase `Workbook` del paquete `openpyxl`](https://openpyxl.readthedocs.io/en/stable/api/openpyxl.workbook.workbook.html).
* Sobre los [bucles anidados](https://www.w3schools.com/python/gloss_python_for_nested.asp). No se aconseja su uso debido a que añaden complejidad al código, pero a veces no queda otro remedio que usarlos.

In [ ]:
wb = Workbook()
ws = wb.active

headers = ['nombre', 'apellidos', 'email', 'grupo']
ws.append(headers)

for group in students.values():
    for student in group:
        ws.append(student)

wb.save('alumnos.xlsx')

if 'alumnos.xlsx' in listdir():
    print('El fichero excel se ha guardado correctamente.')

Ahora podemos abrir el fichero `'alumnos.xlsx'` para comprobar que la información se ha trasladado correctamente a nuestra nueva base de datos.